## Testes Iniciais dos Algoritimos de Chave Fonética

Bibliotecas utilizadas

- NTLK 
- Bibliotecas com implemetações próprias:
    - Algoritimo da seguradora
    - Buscabr 
    - MetaphoneBR
- Soundex 
    - Instalar no Python as bibliotecas soundex e silpa_common

In [1]:

from foneticaBR import foneticaBR
from buscabr import buscaBR
from soundex import Soundex
from metaphoneBR import metaphoneBR
import nltk
from nltk.stem.snowball import SnowballStemmer
import pandas  as pd
from unicodedata import normalize
import numpy as np

In [2]:
chaveRoberto = foneticaBR()
chavebr = buscaBR()
chavemeta = metaphoneBR()
chavesoundex = Soundex()


### Importação dos algoritimos RSLP e Snowball (Português) - NLTK 

In [3]:
chavesoundex = Soundex()
stemmerRSLP = nltk.stem.RSLPStemmer()
stemmersnow = SnowballStemmer("portuguese")

## Importação do arquivo de nomes com as chaves

In [4]:
nomes_com_chave = pd.read_csv('nomes_com_chaves_ws.csv')

In [5]:
nomes_com_chave.head(20)

,NOME,BUSCABR_SEM_VOGAL,BUSCABR,ROBERTO,METAPHONEBR,RSLP,SNOW,CHAVESOUNDEX
0,ABDALA,BDR,ABDARA,ABDAL,ABDL,ABDAL,ABDAL,A134
1,ABDIAS,BD,ABDIA,BDIAS,ABDS,ABD,ABDI,A132
2,ABEILARD,BRD,ABEIRARD,BLARD,ABLRD,ABEILARD,ABEILARD,A1463
3,ABEL,B,ABE,ABL,ABL,ABEL,ABEL,A14
4,ABELA,BR,ABERA,ABL,ABL,ABEL,ABEL,A14
5,ABELARD,BRD,ABERARD,BLARD,ABLRD,ABELARD,ABELARD,A1463
6,ABELINO,BRM,ABERIMO,BLNO,ABLN,ABELIN,ABELIN,A145
7,ABIA,B,ABIA,ABIA,AB,ABI,ABI,A1
8,ABIANOR,BM,ABIAMO,BIANOR,ABN2,ABIAN,ABIANOR,A156
9,ABIDIAS,BD,ABIDIA,BDIAS,ABDS,ABID,ABID,A132


## Processamento dos Testes

In [49]:
# Criação do Bagging

def bagging(listabag,limite):
    
    count = {}
    
    listafinal = []
    
    if len(listabag) > 0:
    
        for l in listabag:

            for i in range(len(l)):

                if count.has_key(l[i]):

                    count[l[i]] = count[l[i]] + 1 

                else:     

                    count[l[i]] = 1 

        for k ,v in count.iteritems():

            if v >= limite: 

                listafinal.append(k) 

    return listafinal

# Função para realização das busca de uma palavra utilizando todas as chaves fonéticas

def busca_nome(palavra):     
    
    listabagging = []
    
    chaveFonetica = chavemeta.chaveMetaphoneBR(palavra)
    resultadoMPBR =nomes_com_chave[nomes_com_chave['METAPHONEBR']==chaveFonetica].NOME
    
    chaveFonetica =  chaveRoberto.chavefonetica(palavra)
    resultadoRoberto = nomes_com_chave[nomes_com_chave['ROBERTO']==chaveFonetica.strip()].NOME  
    
    chaveFonetica = stemmerRSLP.stem(palavra.decode('utf-8')).upper()
    resultadoRSLP = nomes_com_chave[nomes_com_chave['RSLP']==chaveFonetica].NOME  
    
    chaveFonetica = chavesoundex.soundex(palavra.decode('utf-8'))
    resultadoSOUND = nomes_com_chave[nomes_com_chave['CHAVESOUNDEX']==chaveFonetica].NOME  
    
    chaveFonetica = chavebr.chaveBR(palavra,False)
    resultadoBBR = nomes_com_chave[nomes_com_chave['BUSCABR']==chaveFonetica].NOME  
    
    chaveFonetica = chavebr.chaveBR(palavra,True)
    resultadoBBRSV = nomes_com_chave[nomes_com_chave['BUSCABR_SEM_VOGAL']==chaveFonetica].NOME  
    
    chaveFonetica = stemmersnow.stem(palavra.decode('utf-8')).upper()
    resultadoSNOW = nomes_com_chave[nomes_com_chave['SNOW']==chaveFonetica].NOME      

    #listabagging.append(resultadoRSLP.iloc[:].values)
    #listabagging.append(resultadoSNOW.iloc[:].values)
    #listabagging.append(resultadoSOUND.iloc[:].values)
    listabagging.append(resultadoBBRSV.iloc[:].values)
    listabagging.append(resultadoBBR.iloc[:].values)
    listabagging.append(resultadoMPBR.iloc[:].values)
    listabagging.append(resultadoRoberto.iloc[:].values)

    
    resultadoBagging = bagging(listabagging,3) 
    
    data = {"RSLP":{"NOMESENCONTRADOS":list(resultadoRSLP.iloc[:].values)},\
            "SNOW":{"NOMESENCONTRADOS":list(resultadoSNOW.iloc[:].values)},\
            "CHAVESOUNDEX":{"NOMESENCONTRADOS":list(resultadoSOUND.iloc[:].values)},\
            "BUSCABR_SEM_VOGAL":{"NOMESENCONTRADOS":list(resultadoBBRSV.iloc[:].values)},\
            "BUSCABR":{"NOMESENCONTRADOS":list(resultadoBBR.iloc[:].values)},\
            "METAPHONEBR":{"NOMESENCONTRADOS":list(resultadoMPBR.iloc[:].values)},\
            "ROBERTO":{"NOMESENCONTRADOS":list(resultadoRoberto.iloc[:].values)},\
            "BAGGING":{"NOMESENCONTRADOS":list(resultadoBagging)}
            } 
    return data


In [46]:
# Calculo das métricas

def contar_ocorrencias(busca,esperado,nomesEncontrados):
    acertos = 0
    acertosl = 0
    fp = 0
    fn = 0
    fpl= 0
    fnl = 0
    nomesEncontradosL = []
    
    for index,item in enumerate(nomesEncontrados):
        if nltk.edit_distance(item, busca) <=2 :
            nomesEncontradosL.append(item)
        
    # Calcular Falso negativos
    for index,item in enumerate(esperado):
        if item in nomesEncontrados:
            acertos += 1
        else:
            fn += 1
           
    
    # Calcular Falso negativos levestein
    for index,item in enumerate(esperado):
        if item in nomesEncontradosL:
            acertosl += 1
        else:
            fnl += 1
               
    
    #calcular falsos positivos
    for index,item in enumerate(nomesEncontrados):
        if item not in esperado:
            fp += 1
                       
    #calcular falsos positivos levestein
    for index,item in enumerate(nomesEncontradosL):
        if item not in esperado:
            fpl += 1
           
    #print('Esperado: ', esperado , ' Encontrados: ' , nomesEncontrados , ' Acertos: ' , str(acertos), 'FP',fp,'FN',fn )     
    return acertos,acertosl,fp,fn,fpl,fnl

def calculaF1(acertos,fp,fn):

    try:
        precision = float(acertos)/(acertos+fp)
        recall = float(acertos)/(acertos+fn)
        return float(2*(precision*recall)/(precision+recall))
    except:
        return 0.0

def calcula(nroarq):
    records = []

    metricas = 8
    
    sumacertos = list(np.zeros((metricas,), dtype=np.int))
    sumacertosl = list(np.zeros((metricas,), dtype=np.int))
    sumfp = list(np.zeros((metricas,), dtype=np.int))
    sumfn = list(np.zeros((metricas,), dtype=np.int))
    sumfpl = list(np.zeros((metricas,), dtype=np.int))
    sumfnl = list(np.zeros((metricas,), dtype=np.int))

    for index,row in buscasdf.iterrows():
        busca = row[0]
        data = busca_nome(busca)
        resultado = row[1].split(',')

        #Dados de cada algoritmo
        acertos = list(np.zeros((metricas,), dtype=np.int))
        acertosl = list(np.zeros((metricas,), dtype=np.int))
        fp = list(np.zeros((metricas,), dtype=np.int))
        fn = list(np.zeros((metricas,), dtype=np.int))
        fpl = list(np.zeros((metricas,), dtype=np.int))
        fnl = list(np.zeros((metricas,), dtype=np.int))
        f1 = list(np.zeros((metricas,), dtype=np.int))
        #RSLP
        nomesEncontrados = data['RSLP']['NOMESENCONTRADOS']
        acertos[0],acertosl[0],fp[0],fn[0],fpl[0],fnl[0] = contar_ocorrencias(busca,resultado,nomesEncontrados)
        sumacertos[0] += acertos[0]
        sumacertosl[0] += acertosl[0]
        sumfp[0] += fp[0]
        sumfn[0] += fn[0]
        sumfpl[0] += fpl[0]
        sumfnl[0] += fnl[0]

        f1[0] = calculaF1(acertos[0],fp[0],fn[0])

        #SNOWBALL
        nomesEncontrados = data['SNOW']['NOMESENCONTRADOS']
        acertos[1],acertosl[1],fp[1],fn[1],fpl[1],fnl[1] = contar_ocorrencias(busca,resultado,nomesEncontrados)
        sumacertos[1] += acertos[1]
        sumacertosl[1] += acertosl[1]
        sumfp[1] += fp[1]
        sumfn[1] += fn[1]
        sumfpl[1] += fpl[1]
        sumfnl[1] += fnl[1]
        f1[1] = calculaF1(acertos[1],fp[1],fn[1])

        #SOUNDEX
        nomesEncontrados = data['CHAVESOUNDEX']['NOMESENCONTRADOS']
        acertos[2],acertosl[2],fp[2],fn[2],fpl[2],fnl[2] = contar_ocorrencias(busca,resultado,nomesEncontrados)
        sumacertos[2] += acertos[2]
        sumacertosl[2] += acertosl[2]
        sumfp[2] += fp[2]
        sumfn[2] += fn[2]
        sumfpl[2] += fpl[2]
        sumfnl[2] += fnl[2]
        f1[2] = calculaF1(acertos[2],fp[2],fn[2])

        #BUSCABR_SEM_VOGAL
        nomesEncontrados = data['BUSCABR_SEM_VOGAL']['NOMESENCONTRADOS']
        acertos[3],acertosl[3],fp[3],fn[3],fpl[3],fnl[3] = contar_ocorrencias(busca,resultado,nomesEncontrados)
        sumacertos[3] += acertos[3]
        sumacertosl[3] += acertosl[3]
        sumfp[3] += fp[3]
        sumfn[3] += fn[3]
        sumfpl[3] += fpl[3]
        sumfnl[3] += fnl[3]
        f1[3] = calculaF1(acertos[3],fp[3],fn[3])

        #BUSCABR
        nomesEncontrados = data['BUSCABR']['NOMESENCONTRADOS']
        acertos[4],acertosl[4],fp[4],fn[4],fpl[4],fnl[4] = contar_ocorrencias(busca,resultado,nomesEncontrados)
        sumacertos[4] += acertos[4]
        sumacertosl[4] += acertosl[4]
        sumfp[4] += fp[4]
        sumfn[4] += fn[4]
        sumfpl[4] += fpl[4]
        sumfnl[4] += fnl[4]
        f1[4] = calculaF1(acertos[4],fp[4],fn[4])

        #METAPHONE
        nomesEncontrados = data['METAPHONEBR']['NOMESENCONTRADOS']
        acertos[5],acertosl[5],fp[5],fn[5],fpl[5],fnl[5] = contar_ocorrencias(busca,resultado,nomesEncontrados)
        sumacertos[5] += acertos[5]
        sumacertosl[5] += acertosl[5]
        sumfp[5] += fp[5]
        sumfn[5] += fn[5]
        sumfpl[5] += fpl[5]
        sumfnl[5] += fnl[5]
        f1[5] = calculaF1(acertos[5],fp[5],fn[5])

        #ROBERTO
        nomesEncontrados = data['ROBERTO']['NOMESENCONTRADOS']
        acertos[6],acertosl[6],fp[6],fn[6],fpl[6],fnl[6] = contar_ocorrencias(busca,resultado,nomesEncontrados)
        sumacertos[6] += acertos[6]
        sumacertosl[6] += acertosl[6]
        sumfp[6] += fp[6]
        sumfn[6] += fn[6]
        sumfpl[6] += fpl[6]
        sumfnl[6] += fnl[6]
        f1[6] = calculaF1(acertos[6],fp[6],fn[6])

        #BAGGING
        nomesEncontrados = data['BAGGING']['NOMESENCONTRADOS']
        acertos[7],acertosl[7],fp[7],fn[7],fpl[7],fnl[7] = contar_ocorrencias(busca,resultado,nomesEncontrados)
        sumacertos[7] += acertos[7]
        sumacertosl[7] += acertosl[6]
        sumfp[7] += fp[7]
        sumfn[7] += fn[7]
        sumfpl[7] += fpl[7]
        sumfnl[7] += fnl[7]
        f1[7] = calculaF1(acertos[7],fp[7],fn[7])
        
        
        linha ={"BUSCA":busca,"Resultado Esperado": row['Retorno Esperado'],\
                'RSLP': data['RSLP']['NOMESENCONTRADOS'],"ACRSLP": str( acertos[0]),"FPRSLP": str(fp[0]),"FNRSLP":str(fn[0]),"F1RSLP":str(f1[0]),\
                'SNOW': data['SNOW']['NOMESENCONTRADOS'],"ACSNOW": str( acertos[1]),"FPSNOW": str(fp[1]),"FNSNOW":str(fn[1]),"F1SNOW":str(f1[1]),\
                'CHAVESOUNDEX': data['CHAVESOUNDEX']['NOMESENCONTRADOS'],"ACCHAVESOUNDEX": str( acertos[2]),"FPCHAVESOUNDEX": str(fp[2]),"FNCHAVESOUNDEX":str(fn[2]),"F1CHAVESOUNDEX":str(f1[2]),\
                'BUSCABR_SEM_VOGAL': data['BUSCABR_SEM_VOGAL']['NOMESENCONTRADOS'],"ACBUSCABRSV": str( acertos[3]),"FPBUSCABRSV": str(fp[3]),"FNBUSCABRSV":str(fn[3]),"F1BUSCABRSV":str(f1[3]),\
                'BUSCABR': data['BUSCABR']['NOMESENCONTRADOS'],"ACBUSCABR": str( acertos[4]),"FPBUSCABR": str(fp[4]),"FNBUSCABR":str(fn[4]),"F1BUSCABR":str(f1[4]),\
                'METAPHONEBR': data['METAPHONEBR']['NOMESENCONTRADOS'],"ACMETAPHONEBR": str(acertos[5]),"FPMETAPHONEBR": str(fp[5]),"FNMETAPHONEBR":str(fn[5]),"F1METAPHONEBR":str(f1[5]),\
                'ROBERTO': data['ROBERTO']['NOMESENCONTRADOS'],"ACROBERTO": str( acertos[6]),"FPROBERTO": str(fp[6]),"FNROBERTO":str(fn[6]),"F1ROBERTO":str(f1[6]),\
                'BAGGING': data['BAGGING']['NOMESENCONTRADOS'],"ACBAGG": str( acertos[7]),"FPBAGG": str(fp[7]),"FNBAGG":str(fn[7]),"F1BAGG":str(f1[7])}

        records.append(linha)


    #Gerar linha com sumarizacao    
    linha ={"BUSCA":"","Resultado Esperado": "",\
            'RSLP': "","ACRSLP": str( sumacertos[0]),"FPRSLP": str(sumfp[0]),"FNRSLP":str(sumfn[0]),\
            'SNOW': "","ACSNOW": str( sumacertos[1]),"FPSNOW": str(sumfp[1]),"FNSNOW":str(sumfn[1]),\
            'CHAVESOUNDEX': "","ACCHAVESOUNDEX": str( sumacertos[2]),"FPCHAVESOUNDEX": str(sumfp[2]),"FNCHAVESOUNDEX":str(sumfn[2]),\
            'BUSCABR_SEM_VOGAL':"","ACBUSCABRSV": str( sumacertos[3]),"FPBUSCABRSV": str(sumfp[3]),"FNBUSCABRSV":str(sumfn[3]),\
            'BUSCABR': "","ACBUSCABR": str( sumacertos[4]),"FPBUSCABR": str(sumfp[4]),"FNBUSCABR":str(sumfn[4]),\
            'METAPHONEBR': "","ACMETAPHONEBR": str( sumacertos[5]),"FPMETAPHONEBR": str(sumfp[5]),"FNMETAPHONEBR":str(sumfn[5]),\
            'ROBERTO': "","ACROBERTO": str( sumacertos[6]),"FPROBERTO": str(sumfp[6]),"FNROBERTO":str(sumfn[6]),\
            'BAGGING': "","ACBAGG": str( sumacertos[7]),"FPBAGG": str(sumfp[7]),"FNBAGG":str(sumfn[7]) }

    records.append(linha)


    #Gerar metrica F1

    # Os nomes das colunas devem seguir o mesmo padrao por causa da serialização em arquivo do pandas
    linha = {"BUSCA":"Algoritmo","Resultado Esperado":"Precision","RSLP":"Recall","ACRSLP":"F1"}
    records.append(linha)

    algoritmos =["RSLP","SNOWBALL","SOUNDEX","BUSCABR_SEM_VOGAL","BUSCABR","METAPHONEBR","ROBERTO","BAGGING"]

    for x in range(0, 8): 

        precision = float(sumacertos[x])/(sumacertos[x]+sumfp[x])
        recall = float(sumacertos[x])/(sumacertos[x]+sumfn[x])
        f1 = float(2*(precision*recall)/(precision+recall))

        precisionl = float(sumacertosl[x])/(sumacertosl[x]+sumfpl[x])
        recalll = float(sumacertosl[x])/(sumacertosl[x]+sumfnl[x])
        f1l = float(2*(precisionl*recalll)/(precisionl+recalll))   

        linha = {"BUSCA":algoritmos[x],"Resultado Esperado":'{percent:.2%}'.format(percent=precision).replace('.',','),
                 "RSLP":'{percent:.2%}'.format(percent=recall).replace('.',','),"ACRSLP":'{percent:.2%}'.format(percent=f1).replace('.',',')}

        records.append(linha)

        linha = {"BUSCA":algoritmos[x] + " Levenshtein","Resultado Esperado":'{percent:.2%}'.format(percent=precisionl).replace('.',','),
                 "RSLP":'{percent:.2%}'.format(percent=recalll).replace('.',','),"ACRSLP":'{percent:.2%}'.format(percent=f1l).replace('.',',')}

        records.append(linha)    

    df = pd.DataFrame(records,columns=["BUSCA","Resultado Esperado",\
                                       "RSLP","ACRSLP","FPRSLP","FNRSLP","F1RSLP",\
                                       "SNOW","ACSNOW","FPSNOW","FNSNOW","F1SNOW",
                                       "CHAVESOUNDEX","ACCHAVESOUNDEX","FPCHAVESOUNDEX","FNCHAVESOUNDEX","F1CHAVESOUNDEX",
                                       "BUSCABR_SEM_VOGAL","ACBUSCABRSV","FPBUSCABRSV","FNBUSCABRSV","F1BUSCABRSV",
                                       "BUSCABR","ACBUSCABR","FPBUSCABR","FNBUSCABR","F1BUSCABR",
                                       "METAPHONEBR","ACMETAPHONEBR","FPMETAPHONEBR","FNMETAPHONEBR","F1METAPHONEBR",
                                       "ROBERTO","ACROBERTO","FPROBERTO","FNROBERTO","F1ROBERTO",
                                       "BAGGING","ACBAGG","FPBAGG","FNBAGG","F1BAGG"])  



    #Gravando o resultado em CSV
    nomearq = 'testes_automaticos_' + str(nroarq)+ '.csv'
    df.to_csv(path_or_buf=nomearq,index=False,encoding='utf-8',sep=";")    

In [ ]:
qtds = [382]
for nroarq in qtds:
    #Carga do arquivo que contem os nomes para busca e os resultados esperados para cada nome de acordo com a base utilizada
    nome_arq = 'buscas_'+str(nroarq)+'.csv'
    print('Lendo ' + nome_arq)
    buscasdf = pd.read_csv(filepath_or_buffer=nome_arq, sep=';')
    calcula(nroarq)

Lendo buscas_382.csv
